In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
from toolkit.common.constants import *
from toolkit.cars.car_configuration import Car
from toolkit.lap.gps import *
from toolkit.mmd import MMD
from toolkit.steady_state_solver.iterative import Iterative_Solver
from toolkit.common.maths import sa_lut, interpolate

from scipy.optimize import minimize

In [5]:
# Accel Sim Parameters
dt = 0.001 # timestep [s]
L = 75 # length of accel run [m]
rollout = 1 # how far back the transponder is relative to the car front wing [m]

damper_v_front = np.array([0, 0.5, 10]) # damper velocity [in/sec]
damper_f_front = np.array([0, 3.38, 12.82]) # damper force [lbs]
damper_v_rear = np.array([0, 7.86])
damper_f_rear = np.array([0, -116.6])

# DON'T TOUCH:

# unit corrections to meters/sec and N, respectively
damper_v_front *= IN_TO_M
damper_f_front *= LB_TO_KG * G
damper_v_rear *= IN_TO_M
damper_f_rear *= LB_TO_KG * G 

fzfl = 0
fzfr = 0 # initial tire load [N]
fzrl = 0
fzrr = 0

t = 0 # official time [s]
tr = 0 # rollout time [s]

# approximate pitch inertia of the vehicle
I_yy = 0 # TODO: PULL FROM Damper Analysis.ipynb
# TODO: pull other variables from above
# TODO: add vars for unsprung Iyy, k, etc

# state variables
x = -rollout # position [m]
v = 0 # current velocity [m/s]
ax = 0 # current acceleration [m/s/s]
p = 0 # pitch of car, positive pitch is pitch back [rad]
pdot = 0 # pitch rate of the car [rad/s]. positive is pitching backwards, for sake of ease
pddot = 0 # pitch acceleration of the car [rad/s/s]

t_arr = []
kappa_arr = []
kappa_low_arr = []
fx_arr = []
x_arr = []
v_arr = []
a_arr = []
T_arr = []
P_arr = []

In [6]:
car = Car(front_ackermann = "nonlinear", camber_type="combined", toe_type="complexfast", aero_type = "complex")
solver = Iterative_Solver()

In [7]:
def calc_fx(fz, sa, ia, vel):

    def objective(fx_targ):
        # The function returns a tuple, and we need the third output (fx)
        _, _, fx = car.s_r_sel(
            fz,  # Normal force
            sa,                 # Slip angle
            ia,                 # Inclination angle
            vel,                # Speed
            mu_corr=0.65,
            fx_targ=fx_targ
        )
        return -fx  # Minimize negative fx to maximize fx

    upper_bound = car.find_tractive_force(v) / 2 + 1 # add 1 just so the data has less noise when we hit max, idk
    result = minimize_scalar(
        objective,
        bounds=(0, upper_bound),
        method='bounded',
        options={'xatol': 1e-6, 'maxiter': 1000}
    )

    # Extract results
    max_fx = -result.fun  # Convert back to positive
    optimal_fx_targ = result.x
    
    return max_fx, optimal_fx_targ

In [ ]:
def calc_step(car, x, v, fzfl, fzfr, fzrl, fzrr):
    f_max = car.find_tractive_force(v) / 2
    
    # calculate tire conditions
    fzfl, fzfr, fzrl, fzrr, wt_pitch, _ = car.find_contact_patch_loads(long_g=ax, vel=v) # TODO: replace with more advanced function, pull from Damper Analysis.ipynb and include aero sensitivities
    delta_fl, delta_fr, delta_rl, delta_rr = car.calculate_tire_delta_angle(0, wt_pitch=wt_pitch)
    [safl, safr, sarl, sarr] = car.calculate_slip_angles(v, 0, 0, delta_fl, delta_fr, delta_rl, delta_rr)
    ia_fl, ia_fr, ia_rl, ia_rr = car.calculate_ia(0, fzfl, fzfr, fzrl, fzrr, wt_pitch=wt_pitch)
    
    # kappa, bam, fx = car.s_r_sel(fzrl, sarl, ia_rl, v, mu_corr=0.65, fx_targ=f_max)
    
    fx, fx_input = calc_fx(fzrl, sarl, ia_rl, v) # calc best fx
    
    if f_max < fx :
        fx = f_max
        fx_input = fx
    
    kappa, _, fx = car.s_r_sel(fzrl, sarl, ia_rl, v, mu_corr = 0.65, fx_targ = fx_input) # pull kappa of best fx
    kappa_low, _, _ = car.s_r_sel(fzrl, sarl, ia_rl, v, mu_corr = 0.65, fx_targ = fx * 0.99) # get kappa for 99th percentile of fx
    
    fx *= 2 # multiply by 2 bc 2 rear tires
    
    # F = ma --> F/m = a
    ax = fx / car.mass # calc accel
    dv = ax * dt # calc dv
    
    # x = vit + 1/2 at^2
    dx = v * dt + 0.5 * ax * dt * dt
    
    # update state variables
    x += dx
    v += dv
    t += dt
    
    # record keeping
    drag = 0.5 * 1.225 * v*v * car.cd
    t_arr.append(t)
    x_arr.append(x)
    v_arr.append(v)
    a_arr.append(ax)
    kappa_arr.append(kappa)
    kappa_low_arr.append(kappa_low)
    
    torque = (fx+drag) * car.wheel_radius / car.drive_ratio
    omega = v / car.wheel_radius * car.drive_ratio
    power = torque * omega
    T_arr.append(torque) # motor torque
    P_arr.append(power)
    
    
    # now update weight transfer
    # TODO: actually do weight transfer code
    
    # TODO: return accel time
    

In [ ]:
# TODO: calc_step() while loop

In [ ]:
# TODO: nice plots of accel sim